In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv(r"Path\traffic_index_data.csv")

In [3]:
df.head()

,Rank,City,Country,Average Travel Time,Change from 2023,Congestion Percentage,Time Lost Per Year,Congestion Rank
0,260,Santander,Spain,18 min 44 s,no change,20%,35 hours,415
1,472,"Jacksonville, FL",United States of America,10 min 43 s,+ 10 s,20%,26 hours,411
2,184,Nakhon Ratchasima,Thailand,20 min 57 s,no change,33%,65 hours,144
3,221,Cartagena,Spain,19 min 45 s,no change,20%,35 hours,419
4,193,Cambridge,United Kingdom,20 min 41 s,+ 20 s,36%,81 hours,82


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Rank                   500 non-null    int64 
 1   City                   500 non-null    object
 2   Country                500 non-null    object
 3   Average Travel Time    500 non-null    object
 4   Change from 2023       500 non-null    object
 5   Congestion Percentage  500 non-null    object
 6   Time Lost Per Year     500 non-null    object
 7   Congestion Rank        500 non-null    int64 
dtypes: int64(2), object(6)
memory usage: 31.4+ KB


#### Data Transformation will be needed for some columns, Lets do one by one

In [5]:
#Converting Average Travel Time column. This is mixed with minutes and secs, lets keep it simple and just go with minutes

# Function to extract only the minutes
def extract_minutes(time_str):
    return int(time_str.split(" min")[0]) if " min" in time_str else None

# Apply transformation
df["10KM Avg Travel (min)"] = df["Average Travel Time"].apply(extract_minutes)

# Drop the original column
df.drop(columns=["Average Travel Time"], inplace=True)

In [6]:
# Converting "Congestion Percentage" by removing "%" and converting to int
df["Congestion Percentage"] = df["Congestion Percentage"].str.replace("%", "").astype(int)

In [7]:
# Converting "Time Lost Per Year" by removing "hours" and converting to int
df["Hour Lost Per Year"] = df["Time Lost Per Year"].str.replace(" hours", "").astype(int)

# Drop the original column
df.drop(columns=["Time Lost Per Year"], inplace=True)

In [8]:
import re

# Function to convert "Change from 2023" to total seconds
def convert_change_from_2023(change_str):
    if change_str == "no change":
        return 0  # No change, set as 0 seconds
    
    match = re.match(r"([+-]?)\s*(\d+)\s*min\s*(\d*)\s*s?", change_str)  # Match min and sec values
    if match:
        sign = -1 if match.group(1) == "-" else 1  # Determine positive/negative
        minutes = int(match.group(2)) * 60  # Convert minutes to seconds
        seconds = int(match.group(3)) if match.group(3) else 0  # Handle optional seconds
        return sign * (minutes + seconds)  # Apply sign and return total seconds

    match = re.match(r"([+-]?)\s*(\d+)\s*s", change_str)  # Match only seconds
    if match:
        sign = -1 if match.group(1) == "-" else 1
        seconds = int(match.group(2))
        return sign * seconds  # Return as seconds

    match = re.match(r"([+-]?)\s*(\d+)\s*min", change_str)  # Match only minutes
    if match:
        sign = -1 if match.group(1) == "-" else 1
        minutes = int(match.group(2)) * 60
        return sign * minutes  # Convert to seconds

    return None  # Handle unexpected cases

# Apply transformation
df["Change from 2023 (sec)"] = df["Change from 2023"].astype(str).apply(convert_change_from_2023)

# Drop the original column
df.drop(columns=["Change from 2023"], inplace=True)

#### Transformed dataset

In [12]:
df.sample(10)

,Rank,City,Country,Congestion Percentage,Congestion Rank,10KM Avg Travel (min),Hour Lost Per Year,Change from 2023 (sec)
368,1,Barranquilla,Colombia,45,16,36,130,-20
12,392,"Providence, RI",United States of America,31,189,14,53,70
275,498,"Mesa, AZ",United States of America,16,460,9,17,0
101,187,Vigo,Spain,18,438,20,37,20
319,151,Izmir,Turkey,41,31,22,94,-10
47,298,Bielsko-Biala,Poland,29,235,17,52,10
403,259,The Hague,Netherlands,31,204,18,53,30
424,487,"Tulsa, OK",United States of America,13,478,10,13,-10
371,109,Lisbon,Portugal,26,294,23,79,0
301,386,Utrecht,Netherlands,29,242,14,43,30


In [13]:
# Save the cleaned dataset
df.to_csv("cleaned_traffic_index_data.csv", index=False)

print("Transformed Dataset saved!.")

Transformed Dataset saved!.
